# DIVAnd full analysis

This notebook presents the different steps necessary for the creation of a climatology:
1. ODV data reading.
2. Extraction of bathymetry and creation of mask
3. Data download from other sources and duplicate removal.
4. Quality control.
5. Parameter optimisation.
6. Spatio-temporal interpolation with `DIVAnd`.

## Import packages

In [ ]:
using Pkg
Pkg.add(PackageSpec(name="PhysOcean", rev="master"))

In [ ]:
using NCDatasets
using PhysOcean
using Downloads
using DataStructures
using DIVAnd
using Dates
using Statistics
using Random
using Makie, CairoMakie, GeoMakie
include("../config.jl")

## Configuration
* Define the horizontal, vertical (depth levels) and temporal resolutions.
* Select the variable of interest

In [ ]:
dx, dy = 0.125, 0.125
lonr = 11.5:dx:20
latr = 39:dy:46
timerange = [Date(1950,1,1), Date(2017,12,31)];

depthr = [0.,5., 10., 15., 20., 25., 30., 40., 50., 66, 
    75, 85, 100, 112, 125, 135, 150, 175, 200, 225, 250, 
    275, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 
    800, 850, 900, 950, 1000, 1050, 1100, 1150, 1200, 1250, 
    1300, 1350, 1400, 1450, 1500, 1600, 1750, 1850, 2000];
depthr = [0.,10.,20.];

In [ ]:
varname = "Salinity"
yearlist = [1900:2017];
monthlist = [[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]];

We create here the variable TS (for "_time selector_"), which allows us to work with the observations corresponding to each period of interest.

In [ ]:
TS = DIVAnd.TimeSelectorYearListMonthList(yearlist, monthlist);
@show TS;

## Read the ODV file
Adapt the `datadir` and `datafile` values.     
The example is based on a sub-setting of the [Mediterranean Sea](https://www.seadatanet.org/Products#/metadata/cd552057-b604-4004-b838-a4f73cc98fcf) aggregated dataset.      
The dataset has been extracted around the Adriatic Sea and exported to a netCDF using [Ocean Data View](https://odv.awi.de/).
<img src="./Images/MedSeaAggreg.jpg" width="450px">

Download the data files (test and full) if needed.

In [ ]:
datafile = adriaticfile
download_check(datafile, adriaticfileURL)

Read the data from the file

In [ ]:
@time obsval, obslon, obslat, obsdepth, obstime, obsid =
    NCODV.load(Float64, datafile, "Water body salinity");

In [ ]:
f = GeoMakie.Figure()
ax = GeoAxis(f[1, 1], dest = "+proj=merc", title = "Observations")
GeoMakie.scatter!(ax, obslon, obslat; markersize = 2, color = :black)
f

Check the extremal values of the observations

In [ ]:
checkobs((obslon, obslat, obsdepth, obstime), obsval, obsid)

## Extract the bathymetry
It is used to delimit the domain where the interpolation is performed.

### Choice of bathymetry
Modify `bathname` according to the resolution required.

In [ ]:
bathname = gebco08file
download_check(bathname, gebco08fileURL)

In [ ]:
@time bx, by, b = load_bath(bathname, true, lonr, latr);

In [ ]:
plot_bathy(bx, by, b, xticks = 10.0:2.0:20.0, yticks = 39.0:2:48.0)

### Create mask
* False for sea        
* True for land

In [ ]:
_, _, mask = load_mask(bathname, true, lonr, latr, depthr)

In [ ]:
plot_mask(bx, by, mask[:, :, 1], xticks = 10.0:2.0:20.0, yticks = 39.0:2:48.0)

### Edit the mask
As an example we will remove the Tyrrhenian Sea from the domain.

In [ ]:
grid_bx = [i for i in bx, j in by];
grid_by = [j for i in bx, j in by];

In [ ]:
mask_edit = copy(mask);
sel_mask1 = (grid_by .<= 42.6) .& (grid_bx .<= 14.0);
sel_mask2 = (grid_by .<= 41.2) .& (grid_bx .<= 16.2);
mask_edit = mask_edit .* .!sel_mask1 .* .!sel_mask2;
@show size(mask_edit)

The edited mask now looks like this:

In [ ]:
plot_mask(bx, by, mask_edit[:, :, 1], xticks = 10.0:2.0:20.0, yticks = 39.0:2:48.0)

## Extract data from other sources
As an illustration we use the World Ocean Database, among other possibilities.

### World Ocean Database

In [ ]:
# Configuration
if isfile("email.txt")
    email = strip(read("email.txt", String))
    print("getting email address from email.txt")
end
# Or create the variable here:
email = ""

woddatadir = joinpath(datadir, "AdriaticTest/")
mkpath(woddatadir);
@info(woddatadir);

In [ ]:
# Uncomment the next line if you have to download the data
# WorldOceanDatabase.download(lonr,latr,timerange,varname,email,woddatadir);

The following will download an example file and extract its content in `Diva-Workshops/notebooks/3-Analysis/Adriatic`. 
If this fails, the data has to be downloaded manuary (in [zip format](https://dox.uliege.be/index.php/s/Px6r7MPlpXAePB2) or [tar.gz format](https://dox.uliege.be/index.php/s/Px6r7MPlpXAePB2))

Make sure to have a file at the path `Diva-Workshops/notebooks/3-Analysis/Adriatic/WOD/GLD/ocldb1560025519.12915.GLD.nc`.


In [ ]:
download_check(WODadriaticfile, WODadriaticfileURL)
extractcommand = `tar -C $(woddatadir) -xzf $(WODadriaticfile)`
run(extractcommand);

if isfile(joinpath(woddatadir, "WOD-Adriatic/WOD/GLD", "ocldb1560025519.12915.GLD.nc"))
    @info("Files have been sucessfully decompressed")
else
    @warn("Please decompress the file manually")
end

⌛⌛         
Read the data. This can also take up to a few minutes, depending on the size of the domain.

<div class="alert alert-block alert-warning"> 
⚠️ The WOD observations IDs have to be modified in order to be ingested by the XML generation:<br>      
to this end, we add the EDMO code of the U.S. NODC, which is 1977.
</div>

In [ ]:
@time obsvalwod,obslonwod,obslatwod,obsdepthwod,obstimewod,obsidwod = 
WorldOceanDatabase.load(Float64,joinpath(woddatadir, "WOD-Adriatic/WOD"), "Temperature" ,prefixid = "1977-");
@info("Found $(length(obslatwod)) observations in WOD")

Remove the data outside Adriatic (similar to mask editing)

In [ ]:
sel_data1 = (obslatwod .<= 42.6) .& (obslonwod .<= 14.);
sel_data2 = (obslatwod .<= 41.2) .& (obslonwod .<= 16.2);
ndataremove = sum((sel_data1) .| (sel_data2))
sel_data = .~((sel_data1) .| (sel_data2));

obslatwod = obslatwod[sel_data];
obslonwod = obslonwod[sel_data];
obsdepthwod = obsdepthwod[sel_data];
obstimewod = obstimewod[sel_data];
obsvalwod = obsvalwod[sel_data];
obsidwod = obsidwod[sel_data];

@info("Number of removed WOD data: $ndataremove");
obsidwod[1:5]

In [ ]:
f = Figure()
ax = GeoAxis(f[1,1], dest = "+proj=merc", title="Observations")
heatmap!(ax, collect(lonr), collect(latr), mask_edit[:,:,1], colormap=Reverse(:binary))
scatter!(ax, obslon, obslat; markersize=2, color=:blue, label="SeaDataNet")
scatter!(ax, obslonwod, obslatwod; markersize=2, color=:green, label="World Ocean Database")
Legend(f[1, 2], ax, "Datasets")
f

### Extract from another source (optional)
Add here the code to read data from another file.

## Remove duplicates
⌛ The idea here to remove the duplicates coming from the combination of two datasets: SeaDataNet and World Ocean Database. If one has to perform a duplicate detection on a unique dataset (for instance SeaDataNet only), a similar procedure can be applied, as explained below.

### Criteria 
Some values have to be set for the tolerance concerning the positions, times and values of the observations:
* Horizontal distance: 0.01 degree (about 1km)
* Vertical separation: 0.01 m depth
* Time separation: 1 minute.      
* Salinity difference:  0.01 psu.
Such values can be tested and adapted according to the application, the dataset etc. Once the parameters have been set, the following command allows one to identify the _potential_ duplicates:

In [ ]:
@time dupl = DIVAnd.Quadtrees.checkduplicates(
    (obslon,obslat,obsdepth,obstime), obsval, 
    (obslonwod,obslatwod, obsdepthwod, obstimewod), obsvalwod,
    (0.01,0.01,0.01,1/(24*60)),0.01);

### Duplicate from only one data source

If we need to identify duplicates coming only from the SeaDataNet dataset, the following command can be executed:

In [ ]:
@time duplSDN = DIVAnd.Quadtrees.checkduplicates(
    (obslon, obslat, obsdepth, obstime),
    obsval,
    (0.01, 0.01, 0.01, 1 / (24 * 60)),
    0.01,
);

Find the indices of the possible duplicates:

In [ ]:
index = findall(.!isempty.(dupl));
ndupl = length(index);
pcdupl = round(ndupl / length(obslon) * 100; digits=2);
@info("Number of possible duplicates: $ndupl")
@info("Percentage of duplicates: $pcdupl%")

If you decide to combine the 2 (or more) datasets:

In [ ]:
newpoints = isempty.(dupl);
@info("Number of new points: $(sum(newpoints)))")

In [ ]:
obslon = [obslon; obslonwod[newpoints]];
obslat = [obslat; obslatwod[newpoints]];
obsdepth = [obsdepth; obsdepthwod[newpoints]];
obstime = [obstime; obstimewod[newpoints]];
obsval = [obsval; obsvalwod[newpoints]];
obsid = [obsid; obsidwod[newpoints]];

### Create a plot 
To show the additional data points

In [ ]:
f = GeoMakie.Figure()
ax = GeoAxis(f[1,1], dest = "+proj=merc", title="Observations")
GeoMakie.heatmap!(ax, collect(lonr), collect(latr), mask_edit[:,:,1], colormap=Reverse(:binary))
GeoMakie.scatter!(ax, obslon, obslat; markersize=2, color=:blue, label="SeaDataNet")
GeoMakie.scatter!(ax, obslonwod[newpoints], obslatwod[newpoints]; markersize=2, color=:green, 
    label="Additional data\nfrom World Ocean Database")
GeoMakie.Legend(f[1, 2], ax, "Datasets")
f

## Quality control
We check the salinity value.     
Adapt the criteria to your region and variable.

In [ ]:
sel = (obsval .<= 40) .& (obsval .>= 25);

In [ ]:
obsval = obsval[sel]
obslon = obslon[sel]
obslat = obslat[sel]
obsdepth = obsdepth[sel]
obstime = obstime[sel]
obsid = obsid[sel];

## Analysis parameters
### Modify data weight 

The new weights are computed in order to take into account the distance between points.    
<div class="alert alert-block alert-info">
ℹ️ If the dataset is large, this can take a few minutes. 
</div>

The maximal and mean values provide an indication of the spatial proximity between the data.    
If you apply this technique, you need to adapt `epsilon2` as `epsilon2 = epsilon2 * rdiag`.

In [ ]:
@time rdiag = 1.0 ./ DIVAnd.weight_RtimesOne((obslon, obslat), (0.03, 0.03));
@show maximum(rdiag), mean(rdiag)

### Correlation lengths and noise-to-signal ratio
We will use the function `diva3D` for the calculations.     
With this function, the correlation length has to be defined in meters, not in degrees.

In [ ]:
sz = (length(lonr), length(latr), length(depthr));
lenx = fill(100_000.0, sz)   # 100 km
leny = fill(100_000.0, sz)   # 100 km
lenz = fill(25.0, sz);      # 25 m 
len = (lenx, leny, lenz);
epsilon2 = 0.1;
#epsilon2 = epsilon2 * rdiag;

### Output file name

In [ ]:
outputdir = "./Adriatic/output/"
if !isdir(outputdir)
    mkpath(outputdir)
end
filename = joinpath(outputdir, "Water_body_$(replace(varname," "=>"_"))_Adriatic.4Danl.nc")

## Metadata and attributes
Edit the different fields according to the project, the authors etc.     
This is used for the netCDF file but also for the XML needed for the Sextant catalog.

In [ ]:
metadata = OrderedDict(
    # Name of the project (SeaDataCloud, SeaDataNet, EMODNET-chemistry, ...)
    "project" => "SeaDataCloud",

    # URN code for the institution EDMO registry,
    # e.g. SDN:EDMO::1579
    "institution_urn" => "SDN:EDMO::1579",

    # Production group
    #"production" => "Diva group",

    # Name and emails from authors
    "Author_e-mail" =>
        ["Your Name1 <name1@example.com>", "Other Name <name2@example.com>"],

    # Source of the observation
    "source" => "observational data from SeaDataNet and World Ocean Atlas",

    # Additional comment
    "comment" => "Duplicate removal applied to the merged dataset",

    # SeaDataNet Vocabulary P35 URN
    # http://seadatanet.maris2.nl/v_bodc_vocab_v2/search.asp?lib=p35
    # example: SDN:P35::WATERTEMP
    "parameter_keyword_urn" => "SDN:P35::EPC00001",

    # List of SeaDataNet Parameter Discovery Vocabulary P02 URNs
    # http://seadatanet.maris2.nl/v_bodc_vocab_v2/search.asp?lib=p02
    # example: ["SDN:P02::TEMP"]
    "search_keywords_urn" => ["SDN:P02::PSAL"],

    # List of SeaDataNet Vocabulary C19 area URNs
    # SeaVoX salt and fresh water body gazetteer (C19)
    # http://seadatanet.maris2.nl/v_bodc_vocab_v2/search.asp?lib=C19
    # example: ["SDN:C19::3_1"]
    "area_keywords_urn" => ["SDN:C19::3_3"],
    "product_version" => "1.0",
    "product_code" => "something-to-decide",

    # bathymetry source acknowledgement
    # see, e.g.
    # * EMODnet Bathymetry Consortium (2016): EMODnet Digital Bathymetry (DTM).
    # https://doi.org/10.12770/c7b53704-999d-4721-b1a3-04ec60c87238
    # 
    # taken from
    # http://www.emodnet-bathymetry.eu/data-products/acknowledgement-in-publications
    #
    # * The GEBCO Digital Atlas published by the British Oceanographic Data Centre on behalf of IOC and IHO, 2003
    #
    # taken from
    # https://www.bodc.ac.uk/projects/data_management/international/gebco/gebco_digital_atlas/copyright_and_attribution/

    "bathymetry_source" => "The GEBCO Digital Atlas published by the British Oceanographic Data Centre on behalf of IOC and IHO, 2003",

    # NetCDF CF standard name
    # http://cfconventions.org/Data/cf-standard-names/current/build/cf-standard-name-table.html
    # example "standard_name" = "sea_water_temperature",
    "netcdf_standard_name" => "sea_water_salinity",
    "netcdf_long_name" => "sea water salinity",
    "netcdf_units" => "1e-3",

    # Abstract for the product
    "abstract" => "...",

    # This option provides a place to acknowledge various types of support for the
    # project that produced the data
    "acknowledgement" => "...",
    "documentation" => "https://doi.org/doi_of_doc",

    # Digital Object Identifier of the data product
    "doi" => "...",
);

SeaDataNet global attributes:

In [ ]:
ncglobalattrib, ncvarattrib = SDNMetadata(metadata, filename, varname, lonr, latr)

## Analysis
Remove the result file before running the analysis, otherwise you'll get the message     
```julia
NCDatasets.NetCDFError(13, "Permission denied")

```

In [ ]:
if isfile(filename)
    rm(filename) # delete the previous analysis
    @info "Removing file $filename"
end

### Plotting function
Define a plotting function that will be applied for each time index and depth level.     
All the figures will be saved in a selected directory.

In [ ]:
figdir = "./Adriatic/figures/"
isdir(figdir) ? @info("Figure directory already exists") : mkdir(figdir)

In [ ]:
function plotres(timeindex, sel, fit, erri; vmin = 33, vmax = 40)
    tmp = copy(fit)
    nx, ny, nz = size(tmp)

    for i = 1:nz
        fig = Figure()

        # plot the data
        ga = GeoAxis(
            fig[1, 1];
            dest = "+proj=merc",
            title = "Depth: $(depthr[i]) \n Time index: $(timeindex)",
        )
        heatmap!(ga, lonr, latr, mask[:, :, i], colormap = Reverse(:binary))
        hm = heatmap!(
            ga,
            lonr,
            latr,
            tmp[:, :, i],
            interpolate = false,
            colorrange = [vmin, vmax],
        )

        xlims!(ga, (lonr[1], lonr[end]))
        ylims!(ga, (latr[1], latr[end]))
        Colorbar(fig[2, 1], hm, vertical = false)

        # display(fig)
        figname = joinpath(
            figdir,
            varname * "_" * lpad(i, 2, '0') * "_" * lpad(timeindex, 3, '0') * ".png",
        )
        save(figname, fig)
    end
end

### Create the gridded fields using `diva3d`
Here only the noise-to-signal ratio is estimated.     
Set `fitcorrlen` to `true` to also optimise the correlation length.

In [ ]:
@time dbinfo = diva3d(
    (lonr, latr, depthr, TS),
    (obslon, obslat, obsdepth, obstime),
    obsval,
    len,
    epsilon2,
    filename,
    varname,
    bathname = bathname,
    plotres = plotres,
    mask = mask_edit,
    fitcorrlen = false,
    niter_e = 2,
    ncvarattrib = ncvarattrib,
    ncglobalattrib = ncglobalattrib,
    surfextend = true,
);

Example of results: salinity at 20 meters for the April-May-June period.

<img src="./Adriatic/figures/Salinity_02_002.png" width="450px">

### Save the observation metadata

In [ ]:
# DIVAnd.saveobs(filename,(obslon,obslat,obsdepth,obstime),obsid);
DIVAnd.saveobs(
    filename,
    "salinity obs",
    obsval,
    (obslon, obslat, obsdepth, obstime),
    obsid,
    used = dbinfo[:used],
)
#DIVAnd.saveobs(filename, "Oxygen data", obsval, (obslon,obslat,obsdepth,obstime),obsid)

## Apply a posteriori quality control
We can use the structure `dbinfo`.
histogram for quality flags

## XML metadata

For DIVAnd analysis using SeaDataCloud/EMODnet-Chemistry data, one can create a XML description for the product for Sextant

Name of the project:
* "SeaDataCloud" or 
* "EMODNET-chemistry"

In [ ]:
project = "SeaDataCloud";
#project = "EMODNET-chemistry"

Download CDI list

In [ ]:
cdilist = joinpath(datadir, "CDI-list-export.zip")

if !isfile(cdilist)
    Downloads.download("http://emodnet-chemistry.maris2.nl/download/export.zip", cdilist)
end

If `ignore_errors` is `false` (default), then a missing CDI will stop the creatation of the XML metadata.

In [ ]:
ignore_errors = true

# File name based on the variable (but all spaces are replaced by underscores)
xmlfilename = joinpath(outputdir, "Water_body_$(replace(varname," "=>"_")).4Danl.xml")

## Generate a XML file for Sextant catalog
Uncomment the following line of you are using SeaDataCloud or EMODnet-Chemistry data.

<div class="alert alert-block alert-warning"> 
⚠️ This step requires an internet connection. If the code is running on a machine without a connection, then the global and variable attributes have to be prepared before the main run and saved to files. 
</div>

In [ ]:
# divadoxml(filename,varname,project,cdilist,xmlfilename, ignore_errors = ignore_errors)